In [4]:
# Set up CUDA in OS
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import seaborn as sn 
import pandas as pd
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as T
from torchvision import datasets, models, transforms
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import matplotlib.pyplot as plt
import time
import copy

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Check version of Pytorch
print(torch. __version__)

2.1.0


In [6]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [7]:
# Find out if a GPU is available
use_cuda = torch.cuda.is_available()
use_cuda

False

In [9]:
# Set up path for data after downloading
train_images = "C:/Users/bengisubulur/Desktop/Food_Item_Detection.v1i.yolov7pytorch/train/images/"
train_labels = "C:/Users/bengisubulur/Desktop/Food_Item_Detection.v1i.yolov7pytorch/train/labels/"
test_images = "C:/Users/bengisubulur/Desktop/Food_Item_Detection.v1i.yolov7pytorch/test/images/"
test_labels = "C:/Users/bengisubulur/Desktop/Food_Item_Detection.v1i.yolov7pytorch/test/labels/"
valid_images = "C:/Users/bengisubulur/Desktop/Food_Item_Detection.v1i.yolov7pytorch/valid/images/"
valid_labels = "C:/Users/bengisubulur/Desktop/Food_Item_Detection.v1i.yolov7pytorch/valid/labels/"


In [ ]:
# Display image for reference
white_torch = torchvision.io.read_image('C:/Users/Kirudang/Desktop/Assignment 2/Skin cancer dataset/test/benign/1.jpg')
print("This is benign skin cancer")
T.ToPILImage()(white_torch)

In [ ]:
# Display image for reference
wh = torchvision.io.read_image('C:/Users/Kirudang/Desktop/Assignment 2/Skin cancer dataset/test/benign/44.jpg')
print("This is malignant skin cancer")
T.ToPILImage()(wh)

In [ ]:
# Create transform function
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
     transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# Apply for training and test data
train_dataset = datasets.ImageFolder(train_dir, transforms_train)
test_dataset = datasets.ImageFolder(test_dir, transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=12, shuffle=True, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers=0)

In [ ]:
print('Train dataset size:', len(train_dataset))
print('Test dataset size:', len(test_dataset))
class_names = train_dataset.classes
print('Class names:', class_names)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 60
plt.rcParams.update({'font.size': 20})
def imshow(input, title):
    # torch.Tensor => numpy
    input = input.numpy().transpose((1, 2, 0))
    # undo image normalization
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # display images
    plt.imshow(input)
    plt.title(title)
    plt.show()
# load a batch of train image
iterator = iter(train_dataloader)
# visualize a batch of train image
inputs, classes = next(iterator)
out = torchvision.utils.make_grid(inputs[:4])
imshow(out, title=[class_names[x] for x in classes[:4]])